<div style="
  padding: 30px;
  text-align: center;" class='row'>
<div style="float:left;width: 15%;" class='column'><a href="https://www.colombiacompra.gov.co"><img alt="Logo Colombia Compra Eficiente" id="logocce" src="https://www.colombiacompra.gov.co/sites/cce_public/files/files_2020/cce-color.png" style="height: 45px;"></a></div>
    <div style="float:left;width: 70%;" class='column'>
        <h1> Validación de Datos de Bodega
        </h1> 
    </div>
<div style="float:left;width: 15%;" class='column'><a href="https://www.dnp.gov.co/" target="_blank"><img class="float-right" id="logodnp" src="https://www.dnp.gov.co/img/logoNuevo.jpg" style="width: 200px;"></a></div>
</div>

## 2. DESTINO Y AUTORES DEL INFORME / INSUMO

|||
|:--|:--|
|**Destinatario**|<table align='left'><tr><td>*Nombre:*</td> <td>Equipo analítica EMAE</td></tr> <tr><td>*Cargo:*</td> <td>NA</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE</td></tr></table>|
|**Autores**|<table><tr><td>*Nombre:*</td> <td>Equipo de Datos -GAEC</td></tr><tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|
|**Aprobación**|<table><tr><td>*Nombre:*</td> <td>Ricardo Suarez</td></tr> <tr><td>*Cargo:*</td> <td>Subdirector Estudios de Mercado y Abastecimiento Estratégico</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|

In [11]:
### Paquetes usados para la validación de datos

import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import unidecode
import libreria.data_management as dm
from dataprep.clean import clean_duplication
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.0f}'.format

from libreria import login # Lógica para el inicio de sesión a las fuentes de datos

# Cargue de las conexiones a las fuentes de datos
cnxn_sandbox, cursor_sandbox = login.login_sql(login.tipo_bodega.SANDBOX)
cnxn_reports, cursor_reports = login.login_sql(login.tipo_bodega.REPORTS)

In [3]:
# SECOP II
pd.read_sql_query("SELECT TOP 10 * FROM CCE_Reports.SECOPII.V_HistoricoProcedimientos_Depurado", cnxn_reports)

,CompanyName,BuyerVat,PCICode,ProcedureRequestID,Reference,PPI,BuyerDossierID,ProcedureRequestName,ProcedureRequestDescription2,ProfileDocUniqueName,PhaseName,RequestOnlinePublishingDate,Fecha Publicación,Fecha Publicacion fase planeacion precalficacion,Fecha Publicacion fase seleccion precalficacion,Fecha de Menifestacion de Interes,Fecha de publicacion de la fase borrador,Fecha de Publicacion de la fase Seleccion,BasePrice,SubmissionUser,Market,RequestType,ProcedureRequestType,OpenByCategory,LegalFrameworkName,Duration,DurationType,DueDateForReceivingReplies,OpeningRepliesDate,EffectiveOpeningDate,BusinessOperationState,BusinessOperationName,BusinessOperationCountry,AllowAuction,SuppliersInvited,DirectSuppliersInvited,ProcedureVisualizationCount,SuppliersExpressedInterestCount,RepliesCount,ExternalRepliesCount,CounterProposalRepliesCount,DistinctSupplierWithReplies,NumLotes,ProcedureRequestDescription,ProcedureRequestState,ProcedureRequestStateId,Awarded,Award ID,AwardDate,AwardTotalPrice,AwarderDisplayName,SupplierName,SupplierVat,SupplierPhoneNumber,SupplierEmailAddress,DimMainCategoryCode,DimMainCategoryTypeCode,ProcedureRequestOpeningState,TypeOfContract,SubTypeOfContract,AddicionalCategoryCodes,Tipo,CodigoProveedor,ContractNoticeUniqueIdentifier,Departamento Entidad,Ciudad Entidad,Departamento Proveedor,Ciudad Proveedor,Codigo Moneda,NumeroAdendas,AdtEditedAt,OrdenEntidad,Justificación Modalidad de Contratación,Entidad Centralizada,URLProceso,Codigo Entidad,EstadoResumen
0,COLONIA AGRICOLA ACACIAS,822001371,12-08-00-130,CO1.REQ.586746,IPMC-112-CAMIS-218,CO1.BDOS.567853,CO1.BDOS.567853,CONTRATAR EL SUMINISTRO DE ALIMENTO PARA PECES...,CONTRATAR EL SUMINISTRO DE ALIMENTO PARA PECES...,Consulta,Presentación de oferta,2018-10-12 17:19:51.620,2018-10-12 12:19:51.620,None,None,None,None,2018-10-12 12:19:51.620,15140540,SANDRA MILENA DIMATE,Gov.CO,Mínima cuantía,Mínima cuantía,True,None,10,Dias,2018-10-16 14:00:00,2018-10-16 14:10:00,2018-10-16 14:33:09.947,Acacías,GESTIÓN CORPORATIVA,CO,False,34,0,0,0,3,0,0,3,8,CONTRATAR EL SUMINISTRO DE ALIMENTO PARA PECES...,Adjudicado,70,Si,CO1.AWD.378924,2018-10-22 20:42:43.293,9968000.0,SANDRA MILENA DIMATE,ENRUTA TRADE SAS,None,3012668892,contacto@enruta.com.co,V1.10101701,UNSPSC,Cerrado,Compraventa,No Especificado,"V1.10121702, V1.10171501, V1.11101508, V1.1510...",Procesos,704409341,CO1.NTC.563983,Meta,Acacías,Santander,Bucaramanga,COP,0,None,Nacional,Presupuesto inferior al 10% de la menor cuantía,Si,https://community.secop.gov.co/Public/Tenderin...,703704478,Adjudicado
1,COLONIA AGRICOLA ACACIAS,822001371,12-08-00-130,CO1.REQ.586746,IPMC-112-CAMIS-218,CO1.BDOS.567853,CO1.BDOS.567853,CONTRATAR EL SUMINISTRO DE ALIMENTO PARA PECES...,CONTRATAR EL SUMINISTRO DE ALIMENTO PARA PECES...,Consulta,Presentación de oferta,2018-10-12 17:19:51.620,2018-10-12 12:19:51.620,None,None,None,None,2018-10-12 12:19:51.620,15140540,SANDRA MILENA DIMATE,Gov.CO,Mínima cuantía,Mínima cuantía,True,None,10,Dias,2018-10-16 14:00:00,2018-10-16 14:10:00,2018-10-16 14:33:09.947,Acacías,GESTIÓN CORPORATIVA,CO,False,34,0,0,0,3,0,0,3,8,CONTRATAR EL SUMINISTRO DE ALIMENTO PARA PECES...,Adjudicado,70,Si,CO1.AWD.378924,2018-10-22 20:42:43.293,9968000.0,SANDRA MILENA DIMATE,ENRUTA TRADE SAS,None,3012668892,contacto@enruta.com.co,V1.10101701,UNSPSC,Cerrado,Compraventa,No Especificado,"V1.10121702, V1.10171501, V1.11101508, V1.1510...",Procesos,704409341,CO1.NTC.563983,Meta,Acacías,Santander,Bucaramanga,COP,0,None,Nacional,Presupuesto inferior al 10% de la menor cuantía,Si,https://community.secop.gov.co/Public/Tenderin...,703704478,Adjudicado
2,COLONIA AGRICOLA ACACIAS,822001371,12-08-00-130,CO1.REQ.586746,IPMC-112-CAMIS-218,CO1.BDOS.567853,CO1.BDOS.567853,CONTRATAR EL SUMINISTRO DE ALIMENTO PARA PECES...,CONTRATAR EL SUMINISTRO DE ALIMENTO PARA PECES...,Consulta,Presentación de oferta,2018-10-12 17:19:51.620,2018-10-12 12:19:51.620,None,None,None,None,2018-10-12 12:19:51.620,15140540,SANDRA MILENA DIMATE,Gov.CO,Mínima c

In [4]:
# SECOP II
pd.read_sql_query("SELECT TOP 10 * FROM CCE_Reports.SECOPII.V_HistoricoContratos_Depurado", cnxn_reports)

,Nombre Entidad,Código Entidad,NIT Entidad,Orden,Sector,Rama,State,Province,City,Entidad Centralizada,Departamento Entidad,Ciudad Entidad,TipoDocProveedor,Documento Proveedor,Nombre Proveedor,Es Grupo,Es Pyme,Departamento Proveedor,Ciudad Proveedor,RequestReference,Description,JustificationTypeOfContractCode,TypeContract,ContractReference,ContractStartDate,ContractEndDate,DeliverStartDate,DeliverEndDate,Condiciones Entrega,ContractState,AdvancePayments,AdvancePayIndependentProperty,Liquidación,Obligación Ambiental,Obligación de Consumo,Reversión,ContractValue,AdvancedPaymentValue,InvoicedValue,PendingPaymentValue,PaidValue,AmortizedValue,PendingAmortizationValue,PendingExecutionValue,DimMainCategoryCode,ProcedureProfileLabel,AprovalDate,ContractNoticeUniqueIdentifier,Codigo Moneda,Código BPIN,Año BPIN,Estado BPIN,Origen de los Recursos,Destino Gasto,ReferenciaContrato,EsPostConflicto,Saldo CDP,Saldo Vigencia,FechaModificacion,Días Adicionados,Puntos del Acuerdo,Pilares del Acuerdo,URLProceso,AprovalDateDate,Nombre Representante Legal,Nacionalidad Representante Legal,Domicilio Representante Legal,Tipo de Identificación Representante Legal,Identificación Representante Legal,Género Representante Legal,Presupuesto General de la Nacion – PGN,Sistema General de Participaciones,Sistema General de Regalías,"Recursos Propios (Alcaldías, Gobernaciones y Resguardos Indígenas)",Recursos de Credito,Recursos Propios,Codigo Proveedor,Fecha Inicio Liquidacion,Fecha Fin Liquidacion,Objeto Contrato
0,POLICÍA METROPOLITANA DE MANIZALES,701699357,900805219,Nacional,defensa,Ejecutivo,None,Manizales,Caldas,Descentralizada,Distrito Capital de Bogotá,Bogotá,NIT,901145271,Multiservicios Norcasia SAS,No,No,Caldas,Manizales,CO1.BDOS.869047,SUMINISTRO DE COMBUSTIBLES CON DESTINO A LOS V...,Presupuesto inferior al 10% de la menor cuantía,Suministros,CO1.PCCNTR.1038452,2019-08-01 00:00:00,2020-07-31 23:59:00,2019-08-01 00:00:00,2020-07-31 23:59:00,Como acordado previamente,terminado,False,False,Si,No,No,No,5000000.0,0.0,0.0,5000000.0,0.0,0.0,0.0,5000000.0,V1.15101500,Mínima cuantía,2019-07-17 18:53:20.303,CO1.NTC.870706,COP,No Definido,2019,Válido,Presupuesto de entidad nacional,Funcionamiento,91-8-10036-19,No,1500000.000000,2.473681e+10,2022-10-22 18:00:00,0,None,None,https://community.secop.gov.co/Public/Tenderin...,2019-07-17,Rubiel Delgado Idarraga,Colombiano,calle 13 # 3-62,None,10.189.505,None,0.0,0.0,0.0,0.0,0.0,0.0,705716512,2020-08-01 05:00:00,2021-01-12 16:59:00,SUMINISTRO DE COMBUSTIBLES CON DESTINO A LOS V...
1,DIRECCION GENERAL MARITIMA - INRED 4,701806358,806006269,Nacional,defensa,Ejecutivo,None,Cartagena,Bolívar,Centralizada,Bolívar,Cartagena,NIT,890101977,Avantika,No,Si,Distrito Capital de Bogotá,Bogotá,CO1.BDOS.857522,CONTRATAR LA ADQUISICIÓN DE MATERIALES Y REACT...,Presupuesto menor al 10% de la Menor Cuantía,Suministros,CO1.PCCNTR.1038453,2019-07-23 12:00:00,2019-11-27 00:00:00,NaT,NaT,No Definido,Modificado,False,False,Si,No,No,No,212937527.0,0.0,0.0,212937527.0,0.0,0.0,0.0,212937527.0,V1.12141800,Selección Abreviada de Menor Cuantía,2019-12-05 15:24:32.407,CO1.NTC.872307,COP,No Definido,N/D,Válido,Presupuesto de entidad nacional,Funcionamiento,228-GINRED4-2019,No,420825388.000000,0.000000e+00,2019-12-06 12:17:00,39,None,None,https://community.secop.gov.co/Public/Tenderin...,2019-12-05,Ivan Alberto Antequera Castro,colombiano,Bogota,None,8533510,None,NaN,NaN,NaN,NaN,NaN,NaN,701895278,2019-11-27 05:00:00,2020-05-27 05:00:00,CONTRATAR EL SERVICIO DE MANTENIMIENTO PREVENT...
2,CORPORACION AUTONOMA REGIONAL DE CALDAS,704670843,890803005,Nacional,No aplica/No pertenece,Corporación Autónoma,None,None,Caldas,Centralizada,No Definido,No Definido,Cédula de Ciudadanía,10281163,FERRECONDUC y/o WILLIAM HENAO GOMEZ,No,No,Caldas,Manizales,CO1.BDOS.879256,"Suministro de materiales eléctricos, hidrosani...",Presupuesto inferior al 10% de la menor cuantía,Suministros,CO1.PCCNTR.1038454,2019-07-18 12:00:00,2019-12-15 12:00:00,NaT,NaT,A convenir,Modific

In [6]:
## CANTIDAD DE CONTRATOS Y VALOR POR MES DEL AÑO 2023 ----
## El calculo se realiza para optener los contratos unicos tanto en valor como en cantidad.       
## Los datos corresponden con la información reportada por las entidades estatales en el SECOP II 
pd.read_sql_query("SELECT DISTINCT MONTH(VALOR.AprovalDate)	AS 'MES', COUNT(DISTINCT VALOR.ContractReference)	AS 'CONTRATOS', SUM(VALOR.VALOR) AS VALOR FROM (SELECT DISTINCT V.ContractReference, V.AprovalDate, MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE	YEAR(V.AprovalDate) = 2023 AND MONTH(V.AprovalDate) IN (1,2,3,4,5,6) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor') GROUP BY ContractReference, V.AprovalDate) VALOR GROUP BY MONTH(VALOR.AprovalDate) ORDER BY MONTH(VALOR.AprovalDate) ASC", cnxn_reports)

,MES,CONTRATOS,VALOR
0,1,126275,129573025673124
1,2,144824,13940998775595
2,3,83906,150189893801070
3,4,44232,6844707112915
4,5,66548,7503867662581
5,6,103166,17970837184933


In [7]:
# CANTIDAD DE CONTRATOS Y VALOR POR MODALIDAD DE LOS MESES DEL 1 AL 6 DEL AÑO 2023
pd.read_sql_query("SELECT DISTINCT VALOR.ProcedureProfileLabel AS 'MODALIDAD', COUNT(DISTINCT VALOR.ContractReference)	AS 'CONTRATOS', SUM(VALOR.VALOR) AS 'VALOR' FROM (SELECT DISTINCT V.ContractReference, V.ProcedureProfileLabel, MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE	YEAR(V.AprovalDate) = 2023 AND MONTH(V.AprovalDate) IN (1,2,3,4,5,6) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor') GROUP BY ContractReference, V.ProcedureProfileLabel) VALOR GROUP BY VALOR.ProcedureProfileLabel ORDER BY VALOR.ProcedureProfileLabel ASC", cnxn_reports)

,MODALIDAD,CONTRATOS,VALOR
0,None,2799,103613720920
1,CCE-20-Concurso_Meritos_Sin_Lista_Corta_1Sobre,675,629768396613
2,Concurso de méritos abierto,5,6054017155
3,Contratación directa,462304,289880074811118
4,Contratación Directa (con ofertas),5161,6605364280216
5,Contratación régimen especial,73188,10031956470483
6,Contratación régimen especial (con ofertas),3174,3355948146568
7,Enajenación de bienes con sobre cerrado,12,1048216104
8,Enajenación de bienes con subasta,16,1094313975
9,Licitación pública,805,4382537400190


In [14]:
## CANTIDAD DE CONTRATOS Y VALOR POR TIPO DE CONTRATO DE LOS MESES DEL 1 AL 6 DEL AÑO 2023
pd.read_sql_query("SELECT DISTINCT VALOR.TypeContract AS 'MODALIDAD', COUNT(DISTINCT VALOR.ContractReference)	AS 'CONTRATOS', SUM(VALOR.VALOR) AS 'VALOR' FROM (SELECT DISTINCT V.ContractReference, V.TypeContract, MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE	YEAR(V.AprovalDate) = 2023 AND MONTH(V.AprovalDate) IN (1,2,3,4,5,6) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor') GROUP BY ContractReference, V.TypeContract) VALOR GROUP BY VALOR.TypeContract ORDER BY VALOR.TypeContract ASC", cnxn_reports)

,MODALIDAD,CONTRATOS,VALOR
0,Acuerdo Marco de Precios,149,0
1,Arrendamiento de inmuebles,3426,443002752266
2,Arrendamiento de muebles,207,254163403975
3,Asociación Público Privada,25,15311805423
4,Comisión,11,31663498246
5,Comodato,1545,150916562583
6,Compraventa,4005,625723969307
7,Concesión,6,776687158379
8,Consultoría,401,124503741043
9,DecreeLaw092/2017,18381,3704322464451


In [15]:
## TOP 20 - CANTIDAD DE CONTRATOS Y VALOR POR ENTIDAD DE LOS MESES DEL 1 AL 6 DEL AÑO 2023
pd.read_sql_query("SELECT DISTINCT TOP 20 VALOR.[Nombre Entidad] AS 'ENTIDAD', COUNT(DISTINCT VALOR.ContractReference)	AS 'CONTRATOS', SUM(VALOR.VALOR) AS 'VALOR' FROM (SELECT DISTINCT V.ContractReference, V.[Nombre Entidad], MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE	YEAR(V.AprovalDate) = 2023 AND MONTH(V.AprovalDate) IN (1,2,3,4,5,6) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor') GROUP BY ContractReference, V.[Nombre Entidad]) VALOR GROUP BY VALOR.[Nombre Entidad] ORDER BY SUM(VALOR.VALOR) DESC", cnxn_reports)

,ENTIDAD,CONTRATOS,VALOR
0,MUNICIPIOS ASOCIADOS DE LA SUBREGION DE EMBALS...,42,142079074746727
1,DISPENSARIO MEDICO ORIENTE,194,120363360179258
2,DISTRITO ESPECIAL DE CIENCIA TECNOLOGIA E INNO...,1209,3136337754918
3,INSTITUTO DE DESARROLLO URBANO,1586,1837488756795
4,DEPARTAMENTO DE ANTIOQUIA,1021,1691272968141
5,RNEC,262,1178253130817
6,CORPORACION AUTONOMA REGIONAL DE SANTANDER,982,1172426215594
7,UNIDAD NACIONAL DE PROTECCIÓN,1529,1041623692384
8,FONDO DE PASIVO SOCIAL DE FERROCARRILES NACION...,456,893191336004
9,DEPARTAMENTO DE SANTANDER,3800,833075575689


In [16]:
## DATOS DE LA ENTIDAD (Secretaría Distrital de Integración Social), CODIGO 702271321
## CANTIDAD DE CONTRATOS Y VALOR POR MES DEL AÑO 2023
pd.read_sql_query("SELECT DISTINCT MONTH(VALOR.AprovalDate)	AS 'MES', COUNT(DISTINCT VALOR.ContractReference)	AS 'CONTRATOS', SUM(VALOR.VALOR) AS VALOR FROM (SELECT DISTINCT V.ContractReference, V.AprovalDate, MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE	YEAR(V.AprovalDate) = 2023 AND MONTH(V.AprovalDate) IN (1,2,3,4,5,6) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor') AND [Código Entidad] = '702271321' GROUP BY ContractReference, V.AprovalDate) VALOR GROUP BY MONTH(VALOR.AprovalDate) ORDER BY MONTH(VALOR.AprovalDate) ASC", cnxn_reports)

,MES,CONTRATOS,VALOR
0,1,1258,52197594286
1,2,2630,389789243397
2,3,1799,83788371310
3,4,1052,95979620990
4,5,808,32909239408
5,6,800,145271487067


In [17]:
## CANTIDAD DE CONTRATOS Y VALOR POR MODALIDAD DE LOS MESES DEL 1 AL 6 DEL AÑO 2023
pd.read_sql_query("SELECT DISTINCT VALOR.ProcedureProfileLabel AS 'MODALIDAD', COUNT(DISTINCT VALOR.ContractReference)	AS 'CONTRATOS',	SUM(VALOR.VALOR) AS 'VALOR' FROM (SELECT DISTINCT V.ContractReference, V.ProcedureProfileLabel, MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE	YEAR(V.AprovalDate) = 2023 AND MONTH(V.AprovalDate) IN (1,2,3,4,5,6) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor')  AND [Código Entidad] = '702271321' GROUP BY ContractReference, V.ProcedureProfileLabel) VALOR GROUP BY VALOR.ProcedureProfileLabel ORDER BY VALOR.ProcedureProfileLabel ASC", cnxn_reports)

,MODALIDAD,CONTRATOS,VALOR
0,None,1,46998900
1,Contratación directa,8205,678594848755
2,Contratación régimen especial,7,37301694299
3,Contratación régimen especial (con ofertas),121,78370715386
4,Mínima cuantía,4,326155000
5,Selección Abreviada de Menor Cuantía,3,2153714028
6,Selección abreviada subasta inversa,6,3141430090


In [18]:
## CANTIDAD DE CONTRATOS Y VALOR POR TIPO DE CONTRATO DE LOS MESES DEL 1 AL 6 DEL AÑO 2023
pd.read_sql_query("SELECT DISTINCT VALOR.TypeContract AS 'MODALIDAD', COUNT(DISTINCT VALOR.ContractReference) AS 'CONTRATOS', SUM(VALOR.VALOR) AS 'VALOR' FROM (SELECT DISTINCT V.ContractReference, V.TypeContract, MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE	YEAR(V.AprovalDate) = 2023 AND MONTH(V.AprovalDate) IN (1,2,3,4,5,6) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor')  AND [Código Entidad] = '702271321' GROUP BY ContractReference, V.TypeContract) VALOR GROUP BY VALOR.TypeContract ORDER BY VALOR.TypeContract ASC", cnxn_reports)

,MODALIDAD,CONTRATOS,VALOR
0,Arrendamiento de inmuebles,90,11797574589
1,Comodato,2,0
2,DecreeLaw092/2017,121,78370715386
3,No Especificado,1,46998900
4,Otro,39,426370048861
5,Prestación de servicios,8092,281244135620
6,Seguros,1,1599632102
7,Suministros,1,506451000


In [19]:
## TOP 20 - CANTIDAD DE CONTRATOS Y VALOR POR ENTIDAD DE LOS MESES DEL 1 AL 6 DEL AÑO 2023
pd.read_sql_query("SELECT DISTINCT TOP 20 VALOR.[Nombre Entidad] AS 'ENTIDAD', COUNT(DISTINCT VALOR.ContractReference)	AS 'CONTRATOS',	SUM(VALOR.VALOR) AS 'VALOR' FROM (SELECT DISTINCT V.ContractReference, V.[Nombre Entidad], MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE	YEAR(V.AprovalDate) = 2023 AND MONTH(V.AprovalDate) IN (1,2,3,4,5,6) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor')  AND [Código Entidad] = '702271321' GROUP BY ContractReference, V.[Nombre Entidad]) VALOR GROUP BY VALOR.[Nombre Entidad] ORDER BY SUM(VALOR.VALOR) DESC", cnxn_reports)

,ENTIDAD,CONTRATOS,VALOR
0,(Secretaría Distrital de Integración Social),8347,799935556458
